In [2]:
from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt

import dash
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd
import os
import numpy as np
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import shapefile
import descartes
from shapely.geometry import Point
import gmaps
import gmaps.datasets
import geopandas
import plotly.graph_objects as go # or plotly.express as px
import plotly.express as px
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.figure_factory as ff
import re
from datetime import datetime as dt
import json
import dash_bootstrap_components as dbc

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     counties = json.load(response)

In [3]:
df = pd.read_csv('/Users/jiaqilu/Desktop/ENV19/data_visualization/GHC_sample_waterquality.csv')

/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.loc[df['var'] == 'XCO2',:]
df = df[['date','value','county','fips','var']]

In [7]:


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Button('Weekly average', id='Weekly', n_clicks=0),
    html.Button('Monthly average', id='Monthly', n_clicks=0),
    html.Button('Yearly average', id='Yearly', n_clicks=0),
    html.Div(id='container-button-timestamp'),
    
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(2015, 1, 1),
        max_date_allowed=dt(2020, 7, 1),
        initial_visible_month=dt(2019, 6, 14),
        date=str(dt(2019, 6, 14))
    ),
    
    
    html.Div([
        dcc.Graph(id='the_graph')
    ]),
    
    
    dcc.Dropdown(id="slct_var",
                 options=[
                     {"label": "Greenhouse Gas - XCO2", "value": "XCO2"},
                     {"label": "Greenhouse Gas - XCH4", "value": "XCH4"},
                     {"label": "Air Quality - NO2", "value": "NO2"},
                     {"label": "Air Quality - PM2.5", "value": "PM2.5"},
                     {"label": "Water Quality - Dissolved Oxygen", "value": 'Dissolved oxygen (DO)'},
                     {"label": "Water Quality - Orthophosphate", "value": "Orthophosphate"},
                     {"label": "Water Quality - pH", "value": "pH"},
                     {"label": "Water Quality - Sulfate", "value": "Sulfate"},
                     {"label": "Water Quality - Chloride", "value": "Chloride"},
                     {"label": "COVID Cases", "value": "COVID Cases"}
                 ],
                 multi=False,
                 value="XCH4",
                 style={'width': "40%"}
                 ),

    html.Div(id="dropdown-content")
    
])



# Weekly Monthly Yearly Graph=======================

@app.callback(Output(component_id='the_graph', component_property='figure'),
              #Output('container-button-timestamp', 'children'),
              [Input('Weekly', 'n_clicks'),
               Input('Monthly', 'n_clicks'),
               Input('Yearly', 'n_clicks'),
              Input('my-date-picker-single', 'date'),
               Input('slct_var','value')
              
              ])
def displayClick(btn1, btn2, btn3,date, option_var):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    temp = df.loc[df['var'] == 'XCO2']
    date = dt.strptime(re.split('T| ', date)[0], '%Y-%m-%d')
    week_num = date.isocalendar()[1]
    temp['date'] = pd.to_datetime(temp['date'])
    temp['week'] = temp['date'].dt.week
    
    if 'Weekly' in changed_id:
        temp  = temp.loc[((temp['week'] == week_num) & (temp['date'].dt.month == date.month) &(temp['date'].dt.year == date.year)),:]
        print(temp.shape)
    elif 'Monthly' in changed_id:
        temp  = temp.loc[((temp['date'].dt.month == date.month)& (temp['date'].dt.year == date.year)),:]
        print(temp.shape)
    elif 'Yearly' in changed_id:
        temp  = temp.loc[temp['date'].dt.year == date.year,:]
        print(temp.shape)
    
    temp['avg'] = temp['value'].groupby([temp['fips']]).transform('mean')
    temp = temp.drop_duplicates(subset=['fips','avg'],keep = 'first')  
    print(temp)    
    
    
    trace= go.Choroplethmapbox(geojson = counties,
                               locations = temp['fips'],
                               z = temp['avg'],
                               colorscale='hot',
                               reversescale = True,
                               colorbar_thickness=20,
                               text = temp['county'],
                               marker_line_color='white',
                               customdata = temp['fips'],
                               hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                                 '<br> <b>Val </b>: %{z}<br>'+ '<extra></extra>' 
                            )
    
    fig= go.Figure(data = trace)
    mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'

    fig.update_layout(title_text= 'Texas Heatmap',
                      title_x=0.5, width = 700, height=700,
                      mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                     accesstoken= mapboxt,style = 'basic',
                                     zoom=4.5,
                                   ),
                     clickmode='event+select'
                     );

    
    return fig





if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jul/2020 14:44:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 14:44:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 14:44:56] "GET /_dash-dependencies HTTP/1.1" 200 -


           date       value     county     fips   var  week         avg
0    2015-01-04  397.195098      Bowie  48037.0  XCO2     1  402.672447
1    2015-01-04  396.539072       Cass  48067.0  XCO2     1  401.866835
2    2015-01-04  396.734640   Harrison  48203.0  XCO2     1  404.414976
3    2015-01-04  396.369382     Marion  48315.0  XCO2     1  402.067393
4    2015-01-04  395.762778     Morris  48343.0  XCO2     1  405.708273
...         ...         ...        ...      ...   ...   ...         ...
1774 2016-02-04  402.194975   McMullen  48311.0  XCO2     5  407.764113
1916 2016-02-22  396.005300     Dallas  48113.0  XCO2     8  404.462367
2231 2016-04-15  401.673337     Scurry  48415.0  XCO2    15  404.443159
3102 2016-10-03  398.335460     Brazos  48041.0  XCO2    40  401.643857
4005 2017-03-26  404.121492  Childress  48075.0  XCO2    12  408.954055

[254 rows x 7 columns]


127.0.0.1 - - [24/Jul/2020 14:44:59] "POST /_dash-update-component HTTP/1.1" 200 -


(129, 6)
           date       value      county     fips   var  week         avg
7731 2019-06-02  402.967649   Armstrong  48011.0  XCO2    22  405.503127
7732 2019-06-02  409.562145     Briscoe  48045.0  XCO2    22  408.431836
7733 2019-06-02  409.450633      Carson  48065.0  XCO2    22  409.450633
7734 2019-06-02  409.339700      Concho  48095.0  XCO2    22  409.339700
7735 2019-06-02  412.640539     Dickens  48125.0  XCO2    22  410.028580
...         ...         ...         ...      ...   ...   ...         ...
7855 2019-06-29  405.019173  Palo Pinto  48363.0  XCO2    26  405.019173
7856 2019-06-29  405.458025      Parker  48367.0  XCO2    26  405.458025
7857 2019-06-29  405.084934   Somervell  48425.0  XCO2    26  405.084934
7858 2019-06-29  410.489175     Wharton  48481.0  XCO2    26  410.489175
7859 2019-06-29  407.198197     Wichita  48485.0  XCO2    26  407.198197

[105 rows x 7 columns]


...

127.0.0.1 - - [24/Jul/2020 14:46:10] "POST /_dash-update-component HTTP/1.1" 200 -


           date       value     county     fips   var  week         avg
0    2015-01-04  397.195098      Bowie  48037.0  XCO2     1  402.672447
1    2015-01-04  396.539072       Cass  48067.0  XCO2     1  401.866835
2    2015-01-04  396.734640   Harrison  48203.0  XCO2     1  404.414976
3    2015-01-04  396.369382     Marion  48315.0  XCO2     1  402.067393
4    2015-01-04  395.762778     Morris  48343.0  XCO2     1  405.708273
...         ...         ...        ...      ...   ...   ...         ...
1774 2016-02-04  402.194975   McMullen  48311.0  XCO2     5  407.764113
1916 2016-02-22  396.005300     Dallas  48113.0  XCO2     8  404.462367
2231 2016-04-15  401.673337     Scurry  48415.0  XCO2    15  404.443159
3102 2016-10-03  398.335460     Brazos  48041.0  XCO2    40  401.643857
4005 2017-03-26  404.121492  Childress  48075.0  XCO2    12  408.954055

[254 rows x 7 columns]


127.0.0.1 - - [24/Jul/2020 14:46:20] "POST /_dash-update-component HTTP/1.1" 200 -


(1825, 6)
           date       value    county     fips   var  week         avg
7118 2019-01-02  406.844530    Bailey  48017.0  XCO2     1  408.093084
7119 2019-01-02  405.813491   Cochran  48079.0  XCO2     1  408.702085
7120 2019-01-02  407.279575    Parmer  48369.0  XCO2     1  408.394981
7121 2019-01-03  405.443685    Reagan  48383.0  XCO2     1  408.838207
7122 2019-01-04  406.791924  Anderson  48001.0  XCO2     1  408.060134
...         ...         ...       ...      ...   ...   ...         ...
8515 2019-10-21  406.772307   Grayson  48181.0  XCO2    43  406.772307
8690 2019-11-15  408.511057  Angelina  48005.0  XCO2    46  408.511057
8692 2019-11-15  409.211294  Franklin  48159.0  XCO2    46  409.211294
8694 2019-11-15  406.298433    Hardin  48199.0  XCO2    46  406.298433
8705 2019-11-15  407.692922     Tyler  48457.0  XCO2    46  407.692922

[247 rows x 7 columns]
(1825, 6)
           date       value    county     fips   var  week         avg
7118 2019-01-02  406.844530    Ba

127.0.0.1 - - [24/Jul/2020 14:46:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 14:46:27] "POST /_dash-update-component HTTP/1.1" 200 -
